In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score, precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, ExtraTreesClassifier, AdaBoostClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from xgboost import XGBClassifier

### Iteration 1

In [2]:
ch=pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")
ch.drop(["customerID"],axis=1,inplace=True)

ch["TotalCharges"][ch["TotalCharges"]==" "]=0
ch["TotalCharges"]=ch["TotalCharges"].astype("float64")

#ch["tenure"][ch["Contract"]=="Month-to-month"]=0
#ch["tenure"][ch["Contract"]=="One year"]=1
#ch["tenure"][ch["Contract"]=="Two year"]=2

C:\Users\Barath\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [3]:
ch[["gender","Partner","Dependents","PhoneService",'PaperlessBilling',"Churn"]]=ch[["gender","Partner","Dependents","PhoneService",'PaperlessBilling',"Churn"]].apply(LabelEncoder().fit_transform)
#ch[["gender","Partner","Dependents","PhoneService","MultipleLines","InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV",'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',"Churn"]]=ch[["gender","Partner","Dependents","PhoneService","MultipleLines","InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV",'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',"Churn"]].apply(LabelEncoder().fit_transform)

In [4]:
u=["MultipleLines","InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV",'StreamingMovies', 'Contract', 'PaymentMethod']
for i in u:
    m=ch[i].unique()
    m=m+i
    dummy = pd.get_dummies(ch[i],columns=ch[i].unique(),prefix=i)
    ch=pd.concat([ch,dummy],axis=1,sort=True)
    ch.drop([i],axis=1,inplace=True)
ch.drop(['OnlineSecurity_No internet service',"OnlineBackup_No internet service","DeviceProtection_No internet service","TechSupport_No internet service","StreamingTV_No internet service","StreamingMovies_No internet service"],axis=1,inplace=True)

In [5]:
rf= RandomForestClassifier()
bc=BaggingClassifier()
etc=ExtraTreesClassifier()
abc=AdaBoostClassifier()
xgb=XGBClassifier()
lr=LogisticRegression()
knn=KNeighborsClassifier()
nb=GaussianNB()
tree=DecisionTreeClassifier()
#vc=VotingClassifier(estimators=200)
gb= GradientBoostingClassifier()

model=[rf,bc,etc,abc,xgb,lr,knn,nb,tree,gb]
names=["Random forest","Bagging","Extra trees","adaboost","XG Boost","Logistic Regression","K Nearest Neighbors","Decision Tree","Gradient boosting"]

In [6]:
x=ch.drop(["Churn"],axis=1)
y=ch["Churn"]

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.35,random_state=101)

In [7]:
scores=pd.DataFrame(columns=["index","model","train score","accuracy score","recall score","precision score","tn","fp","tp","fn"])
j=0
for i,k in zip(model,names):
    i.fit(xtrain,ytrain)
    pred=i.predict(xtest)
    scores.loc[j]=[j,k,i.score(xtrain,ytrain),accuracy_score(ytest,pred),recall_score(ytest,pred),precision_score(ytest,pred),confusion_matrix(ytest,pred)[0,0],confusion_matrix(ytest,pred)[0,1],confusion_matrix(ytest,pred)[1,1],confusion_matrix(ytest,pred)[1,0]]
    j=j+1
scores

C:\Users\Barath\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


,index,model,train score,accuracy score,recall score,precision score,tn,fp,tp,fn
0,0,Random forest,0.998471,0.783455,0.469697,0.627530,1622,184,310,350
1,1,Bagging,0.979026,0.772506,0.430303,0.605544,1621,185,284,376
2,2,Extra trees,0.998471,0.759530,0.456061,0.562617,1572,234,301,359
3,3,adaboost,0.803583,0.815896,0.536364,0.705179,1658,148,354,306
4,4,XG Boost,0.821936,0.811436,0.533333,0.691552,1649,157,352,308
5,5,Logistic Regression,0.805113,0.809002,0.551515,0.675325,1631,175,364,296
6,6,K Nearest Neighbors,0.830020,0.758313,0.421212,0.565041,1592,214,278,382
7,7,Decision Tree,0.734324,0.742498,0.789394,0.512291,1310,496,521,139
8,8,Gradient boosting,0.998471,0.731144,0.478788,0.497638,1487,319,316,344


## Iteration 2

In [9]:
ch=pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")
ch.drop(["customerID"],axis=1,inplace=True)

ch["TotalCharges"][ch["TotalCharges"]==" "]=0
ch["TotalCharges"]=ch["TotalCharges"].astype("float64")

ch["tenure"][ch["Contract"]=="Month-to-month"]=0
ch["tenure"][ch["Contract"]=="One year"]=1
ch["tenure"][ch["Contract"]=="Two year"]=2

C:\Users\Barath\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Barath\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\Barath\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Barath\Anaconda3\lib\site-packages\ipy

In [10]:
ch[["gender","Partner","Dependents","PhoneService",'PaperlessBilling',"Churn"]]=ch[["gender","Partner","Dependents","PhoneService",'PaperlessBilling',"Churn"]].apply(LabelEncoder().fit_transform)
#ch[["gender","Partner","Dependents","PhoneService","MultipleLines","InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV",'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',"Churn"]]=ch[["gender","Partner","Dependents","PhoneService","MultipleLines","InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV",'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',"Churn"]].apply(LabelEncoder().fit_transform)

In [11]:
u=["MultipleLines","InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV",'StreamingMovies', 'Contract', 'PaymentMethod']
for i in u:
    m=ch[i].unique()
    m=m+i
    dummy = pd.get_dummies(ch[i],columns=ch[i].unique(),prefix=i)
    ch=pd.concat([ch,dummy],axis=1,sort=True)
    ch.drop([i],axis=1,inplace=True)
ch.drop(['OnlineSecurity_No internet service',"OnlineBackup_No internet service","DeviceProtection_No internet service","TechSupport_No internet service","StreamingTV_No internet service","StreamingMovies_No internet service"],axis=1,inplace=True)

In [12]:
x=ch.drop(["Churn"],axis=1)
y=ch["Churn"]

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.35,random_state=101)

In [13]:
scores=pd.DataFrame(columns=["index","model","train score","accuracy score","recall score","precision score","tn","fp","tp","fn"])
j=0
for i,k in zip(model,names):
    i.fit(xtrain,ytrain)
    pred=i.predict(xtest)
    scores.loc[j]=[j,k,i.score(xtrain,ytrain),accuracy_score(ytest,pred),recall_score(ytest,pred),precision_score(ytest,pred),confusion_matrix(ytest,pred)[0,0],confusion_matrix(ytest,pred)[0,1],confusion_matrix(ytest,pred)[1,1],confusion_matrix(ytest,pred)[1,0]]
    j=j+1
scores

C:\Users\Barath\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


,index,model,train score,accuracy score,recall score,precision score,tn,fp,tp,fn
0,0,Random forest,0.998471,0.772912,0.453030,0.600402,1607,199,299,361
1,1,Bagging,0.979026,0.779805,0.448485,0.623158,1627,179,296,364
2,2,Extra trees,0.998471,0.750203,0.448485,0.540146,1554,252,296,364
3,3,adaboost,0.805113,0.809002,0.519697,0.690141,1652,154,343,317
4,4,XG Boost,0.824776,0.812247,0.525758,0.698189,1656,150,347,313
5,5,Logistic Regression,0.798995,0.802514,0.528788,0.664762,1630,176,349,311
6,6,K Nearest Neighbors,0.828927,0.760341,0.425758,0.569980,1594,212,281,379
7,7,Decision Tree,0.723400,0.734388,0.809091,0.502352,1277,529,534,126
8,8,Gradient boosting,0.998471,0.722222,0.481818,0.481089,1463,343,318,342


### Iteration 3

In [14]:
ch=pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")
ch.drop(["customerID"],axis=1,inplace=True)

ch["TotalCharges"][ch["TotalCharges"]==" "]=0
ch["TotalCharges"]=ch["TotalCharges"].astype("float64")

#ch["tenure"][ch["Contract"]=="Month-to-month"]=0
#ch["tenure"][ch["Contract"]=="One year"]=1
#ch["tenure"][ch["Contract"]=="Two year"]=2

C:\Users\Barath\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [15]:
#ch[["gender","Partner","Dependents","PhoneService",'PaperlessBilling',"Churn"]]=ch[["gender","Partner","Dependents","PhoneService",'PaperlessBilling',"Churn"]].apply(LabelEncoder().fit_transform)
ch[["gender","Partner","Dependents","PhoneService","MultipleLines","InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV",'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',"Churn"]]=ch[["gender","Partner","Dependents","PhoneService","MultipleLines","InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV",'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',"Churn"]].apply(LabelEncoder().fit_transform)

In [16]:
x=ch.drop(["Churn"],axis=1)
y=ch["Churn"]

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.35,random_state=101)

In [17]:
scores=pd.DataFrame(columns=["index","model","train score","accuracy score","recall score","precision score","tn","fp","tp","fn"])
j=0
for i,k in zip(model,names):
    i.fit(xtrain,ytrain)
    pred=i.predict(xtest)
    scores.loc[j]=[j,k,i.score(xtrain,ytrain),accuracy_score(ytest,pred),recall_score(ytest,pred),precision_score(ytest,pred),confusion_matrix(ytest,pred)[0,0],confusion_matrix(ytest,pred)[0,1],confusion_matrix(ytest,pred)[1,1],confusion_matrix(ytest,pred)[1,0]]
    j=j+1
scores

C:\Users\Barath\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


,index,model,train score,accuracy score,recall score,precision score,tn,fp,tp,fn
0,0,Random forest,0.998471,0.791971,0.475758,0.652807,1639,167,314,346
1,1,Bagging,0.979463,0.774534,0.439394,0.609244,1620,186,290,370
2,2,Extra trees,0.998471,0.779805,0.469697,0.616302,1613,193,310,350
3,3,adaboost,0.803583,0.809408,0.534848,0.684109,1643,163,353,307
4,4,XG Boost,0.822373,0.808191,0.522727,0.685885,1648,158,345,315
5,5,Logistic Regression,0.803146,0.811841,0.546970,0.686312,1641,165,361,299
6,6,K Nearest Neighbors,0.830238,0.759124,0.415152,0.568465,1598,208,274,386
7,7,Decision Tree,0.752458,0.756691,0.734848,0.532967,1381,425,485,175
8,8,Gradient boosting,0.998471,0.719789,0.496970,0.477438,1447,359,328,332


### Iteration 4

In [18]:
ch=pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")
ch.drop(["customerID"],axis=1,inplace=True)

ch["TotalCharges"][ch["TotalCharges"]==" "]=0
ch["TotalCharges"]=ch["TotalCharges"].astype("float64")

ch["tenure"][ch["Contract"]=="Month-to-month"]=0
ch["tenure"][ch["Contract"]=="One year"]=1
ch["tenure"][ch["Contract"]=="Two year"]=2

C:\Users\Barath\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Barath\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\Barath\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Barath\Anaconda3\lib\site-packages\ipy

In [19]:
#ch[["gender","Partner","Dependents","PhoneService",'PaperlessBilling',"Churn"]]=ch[["gender","Partner","Dependents","PhoneService",'PaperlessBilling',"Churn"]].apply(LabelEncoder().fit_transform)
ch[["gender","Partner","Dependents","PhoneService","MultipleLines","InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV",'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',"Churn"]]=ch[["gender","Partner","Dependents","PhoneService","MultipleLines","InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV",'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',"Churn"]].apply(LabelEncoder().fit_transform)

In [20]:
x=ch.drop(["Churn"],axis=1)
y=ch["Churn"]

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.35,random_state=101)

In [21]:
scores=pd.DataFrame(columns=["index","model","train score","accuracy score","recall score","precision score","tn","fp","tp","fn"])
j=0
for i,k in zip(model,names):
    i.fit(xtrain,ytrain)
    pred=i.predict(xtest)
    scores.loc[j]=[j,k,i.score(xtrain,ytrain),accuracy_score(ytest,pred),recall_score(ytest,pred),precision_score(ytest,pred),confusion_matrix(ytest,pred)[0,0],confusion_matrix(ytest,pred)[0,1],confusion_matrix(ytest,pred)[1,1],confusion_matrix(ytest,pred)[1,0]]
    j=j+1
scores

C:\Users\Barath\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


,index,model,train score,accuracy score,recall score,precision score,tn,fp,tp,fn
0,0,Random forest,0.998471,0.780211,0.459091,0.620902,1621,185,303,357
1,1,Bagging,0.979899,0.774939,0.457576,0.605210,1609,197,302,358
2,2,Extra trees,0.998471,0.757908,0.443939,0.560229,1576,230,293,367
3,3,adaboost,0.806860,0.812652,0.550000,0.687500,1641,165,363,297
4,4,XG Boost,0.821936,0.808191,0.522727,0.685885,1648,158,345,315
5,5,Logistic Regression,0.798121,0.800487,0.540909,0.653846,1617,189,357,303
6,6,K Nearest Neighbors,0.828709,0.760341,0.421212,0.570842,1597,209,278,382
7,7,Decision Tree,0.731484,0.739659,0.783333,0.508858,1307,499,517,143
8,8,Gradient boosting,0.998471,0.730333,0.548485,0.496571,1439,367,362,298


### Iteration - 5

In [22]:
ch=pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")
ch.drop(["customerID"],axis=1,inplace=True)

ch["TotalCharges"][ch["TotalCharges"]==" "]=0
ch["TotalCharges"]=ch["TotalCharges"].astype("float64")

#ch["tenure"][ch["Contract"]=="Month-to-month"]=0
#ch["tenure"][ch["Contract"]=="One year"]=1
#ch["tenure"][ch["Contract"]=="Two year"]=2

C:\Users\Barath\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [23]:
ch[["gender","Partner","Dependents","PhoneService",'PaperlessBilling',"Churn"]]=ch[["gender","Partner","Dependents","PhoneService",'PaperlessBilling',"Churn"]].apply(LabelEncoder().fit_transform)
#ch[["gender","Partner","Dependents","PhoneService","MultipleLines","InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV",'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',"Churn"]]=ch[["gender","Partner","Dependents","PhoneService","MultipleLines","InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV",'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',"Churn"]].apply(LabelEncoder().fit_transform)

In [24]:
u=["MultipleLines","InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV",'StreamingMovies', 'Contract', 'PaymentMethod']
for i in u:
    m=ch[i].unique()
    m=m+i
    dummy = pd.get_dummies(ch[i],columns=ch[i].unique(),prefix=i)
    ch=pd.concat([ch,dummy],axis=1,sort=True)
    ch.drop([i],axis=1,inplace=True)
ch.drop(['OnlineSecurity_No internet service',"OnlineBackup_No internet service","DeviceProtection_No internet service","TechSupport_No internet service","StreamingTV_No internet service","StreamingMovies_No internet service"],axis=1,inplace=True)

In [47]:
rf= RandomForestClassifier()
bc=BaggingClassifier()
etc=ExtraTreesClassifier()
abc=AdaBoostClassifier(base_estimator=lr)
lr=LogisticRegression(max_iter=2000)
xgb=XGBClassifier()
knn=KNeighborsClassifier()
nb=GaussianNB()
tree=DecisionTreeClassifier()
#vc=VotingClassifier(estimators=200)
gb= GradientBoostingClassifier()

model=[rf,bc,etc,abc,xgb,lr,knn,nb,tree,gb]
names=["Random forest","Bagging","Extra trees","adaboost","XG Boost","Logistic Regression","K Nearest Neighbors","Decision Tree","Gradient boosting"]

In [97]:
x=ch.drop(["gender","InternetService_Fiber optic","Churn"],axis=1)
y=ch["Churn"]

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2,random_state=525)

In [98]:
scores=pd.DataFrame(columns=["index","model","train score","accuracy score","recall score","precision score","tn","fp","tp","fn"])
j=0
for i,k in zip(model,names):
    i.fit(xtrain,ytrain)
    pred=i.predict(xtest)
    scores.loc[j]=[j,k,i.score(xtrain,ytrain),accuracy_score(ytest,pred),recall_score(ytest,pred),precision_score(ytest,pred),confusion_matrix(ytest,pred)[0,0],confusion_matrix(ytest,pred)[0,1],confusion_matrix(ytest,pred)[1,1],confusion_matrix(ytest,pred)[1,0]]
    j=j+1
scores

,index,model,train score,accuracy score,recall score,precision score,tn,fp,tp,fn
0,0,Random forest,0.998225,0.783534,0.525773,0.627692,900,121,204,184
1,1,Bagging,0.979233,0.769340,0.474227,0.603279,900,121,184,204
2,2,Extra trees,0.998225,0.759404,0.502577,0.571848,875,146,195,193
3,3,adaboost,0.804579,0.822569,0.615979,0.702941,920,101,239,149
4,4,XG Boost,0.821264,0.798439,0.574742,0.652047,902,119,223,165
5,5,Logistic Regression,0.802094,0.819021,0.621134,0.690544,913,108,241,147
6,6,K Nearest Neighbors,0.830671,0.757984,0.451031,0.577558,893,128,175,213
7,7,Decision Tree,0.736422,0.725337,0.780928,0.500826,719,302,303,85
8,8,Gradient boosting,0.998225,0.716820,0.533505,0.487059,803,218,207,181


In [57]:
x=ch.drop(["gender","InternetService_Fiber optic","Churn"],axis=1)
y=ch["Churn"]



array([[4447,  727],
       [ 742, 1127]], dtype=int64)

In [89]:
recall=pd.DataFrame(columns=["index","score"])

In [99]:
for i in range(1111):
    xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2,random_state=i)
    tree.fit(xtrain,ytrain)
    pred=tree.predict(xtest)
    recall.loc[i]=[i,recall_score(ytest,pred)]    
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [100]:
recall[["index","score"]][recall["score"]==recall["score"].max()]

,index,score
940,940.0,0.588068


In [101]:
recall["score"].max()

0.5880681818181818

In [51]:
ch1=ch.copy()

In [52]:
senior=ch1.groupby("SeniorCitizen")

### Iteration 6 - RFE 

In [6]:
ch=pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")
ch.drop(["customerID"],axis=1,inplace=True)

ch["TotalCharges"][ch["TotalCharges"]==" "]=0
ch["TotalCharges"]=ch["TotalCharges"].astype("float64")

#ch["tenure"][ch["Contract"]=="Month-to-month"]=0
#ch["tenure"][ch["Contract"]=="One year"]=1
#ch["tenure"][ch["Contract"]=="Two year"]=2

C:\Users\Barath\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [7]:
ch[["gender","Partner","Dependents","PhoneService",'PaperlessBilling',"Churn"]]=ch[["gender","Partner","Dependents","PhoneService",'PaperlessBilling',"Churn"]].apply(LabelEncoder().fit_transform)
#ch[["gender","Partner","Dependents","PhoneService","MultipleLines","InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV",'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',"Churn"]]=ch[["gender","Partner","Dependents","PhoneService","MultipleLines","InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV",'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',"Churn"]].apply(LabelEncoder().fit_transform)

u=["MultipleLines","InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV",'StreamingMovies', 'Contract', 'PaymentMethod']
for i in u:
    m=ch[i].unique()
    m=m+i
    dummy = pd.get_dummies(ch[i],columns=ch[i].unique(),prefix=i)
    ch=pd.concat([ch,dummy],axis=1,sort=True)
    ch.drop([i],axis=1,inplace=True)
ch.drop(['OnlineSecurity_No internet service',"OnlineBackup_No internet service","DeviceProtection_No internet service","TechSupport_No internet service","StreamingTV_No internet service","StreamingMovies_No internet service"],axis=1,inplace=True)


In [8]:
rf= RandomForestClassifier()
bc=BaggingClassifier()
etc=ExtraTreesClassifier()
abc=AdaBoostClassifier()
xgb=XGBClassifier()
lr=LogisticRegression()
knn=KNeighborsClassifier()
nb=GaussianNB()
tree=DecisionTreeClassifier()
#vc=VotingClassifier(estimators=200)
gb= GradientBoostingClassifier()

model=[rf,bc,etc,abc,xgb,lr,knn,nb,tree,gb]
names=["Random forest","Bagging","Extra trees","adaboost","XG Boost","Logistic Regression","K Nearest Neighbors","Decision Tree","Gradient boosting"]

In [10]:
x=ch[['PhoneService', 'InternetService_DSL', 'InternetService_Fiber optic',
       'OnlineSecurity_No', 'OnlineBackup_No', 'TechSupport_No',
       'Contract_Month-to-month', 'Contract_Two year',
       'PaymentMethod_Electronic check']]
y=ch["Churn"]

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2)

In [11]:
scores=pd.DataFrame(columns=["index","model","train score","accuracy score","recall score","precision score","tn","fp","tp","fn"])
j=0
for i,k in zip(model,names):
    i.fit(xtrain,ytrain)
    pred=i.predict(xtest)
    scores.loc[j]=[j,k,i.score(xtrain,ytrain),accuracy_score(ytest,pred),recall_score(ytest,pred),precision_score(ytest,pred),confusion_matrix(ytest,pred)[0,0],confusion_matrix(ytest,pred)[0,1],confusion_matrix(ytest,pred)[1,1],confusion_matrix(ytest,pred)[1,0]]
    j=j+1
scores

,index,model,train score,accuracy score,recall score,precision score,tn,fp,tp,fn
0,0,Random forest,0.789847,0.782825,0.5375,0.639881,888,121,215,185
1,1,Bagging,0.789670,0.776437,0.5400,0.622478,878,131,216,184
2,2,Extra trees,0.789847,0.782825,0.5375,0.639881,888,121,215,185
3,3,adaboost,0.787007,0.785664,0.5825,0.633152,874,135,233,167
4,4,XG Boost,0.788782,0.782825,0.5575,0.633523,880,129,223,177
5,5,Logistic Regression,0.788782,0.782825,0.5575,0.633523,880,129,223,177
6,6,K Nearest Neighbors,0.775293,0.753726,0.6200,0.559819,814,195,248,152
7,7,Decision Tree,0.737487,0.738822,0.8050,0.526144,719,290,322,78
8,8,Gradient boosting,0.789847,0.782825,0.5375,0.639881,888,121,215,185


### Using all features Iteration - 7

In [12]:
x=ch[['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'PaperlessBilling', 'MonthlyCharges', 'TotalCharges', 'MultipleLines_No', 'MultipleLines_No phone service', 'MultipleLines_Yes', 'InternetService_DSL', 'InternetService_Fiber optic', 'InternetService_No', 'OnlineSecurity_No', 'OnlineSecurity_Yes', 'OnlineBackup_No', 'OnlineBackup_Yes', 'DeviceProtection_No', 'DeviceProtection_Yes', 'TechSupport_No', 'TechSupport_Yes', 'StreamingTV_No', 'StreamingTV_Yes', 'StreamingMovies_No', 'StreamingMovies_Yes', 'Contract_Month-to-month', 'Contract_One year', 'Contract_Two year', 'PaymentMethod_Bank transfer (automatic)', 'PaymentMethod_Credit card (automatic)', 'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check']]
y=ch["Churn"]

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2)

In [13]:
scores=pd.DataFrame(columns=["index","model","train score","accuracy score","recall score","precision score","tn","fp","tp","fn"])
j=0
for i,k in zip(model,names):
    i.fit(xtrain,ytrain)
    pred=i.predict(xtest)
    scores.loc[j]=[j,k,i.score(xtrain,ytrain),accuracy_score(ytest,pred),recall_score(ytest,pred),precision_score(ytest,pred),confusion_matrix(ytest,pred)[0,0],confusion_matrix(ytest,pred)[0,1],confusion_matrix(ytest,pred)[1,1],confusion_matrix(ytest,pred)[1,0]]
    j=j+1
scores

C:\Users\Barath\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


,index,model,train score,accuracy score,recall score,precision score,tn,fp,tp,fn
0,0,Random forest,0.997693,0.801278,0.486702,0.677778,946,87,183,193
1,1,Bagging,0.981363,0.784244,0.470745,0.627660,928,105,177,199
2,2,Extra trees,0.997693,0.787793,0.481383,0.635088,929,104,181,195
3,3,adaboost,0.810437,0.810504,0.534574,0.686007,941,92,201,175
4,4,XG Boost,0.820376,0.818311,0.537234,0.711268,951,82,202,174
5,5,Logistic Regression,0.805822,0.813343,0.558511,0.684039,936,97,210,166
6,6,K Nearest Neighbors,0.830671,0.763662,0.417553,0.579336,919,114,157,219
7,7,Decision Tree,0.733582,0.746629,0.781915,0.516696,758,275,294,82
8,8,Gradient boosting,0.997693,0.730305,0.478723,0.494505,849,184,180,196


### Iteration - 8 - SFS

In [14]:
x=ch[['SeniorCitizen', 'tenure', 'PhoneService', 'MultipleLines_No', 'MultipleLines_No phone service', 'MultipleLines_Yes', 'InternetService_DSL', 'InternetService_Fiber optic', 'InternetService_No', 'OnlineSecurity_No', 'OnlineSecurity_Yes', 'Contract_Month-to-month', 'Contract_One year', 'Contract_Two year', 'PaymentMethod_Mailed check']]
y=ch["Churn"]

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2)

In [15]:
scores=pd.DataFrame(columns=["index","model","train score","accuracy score","recall score","precision score","tn","fp","tp","fn"])
j=0
for i,k in zip(model,names):
    i.fit(xtrain,ytrain)
    pred=i.predict(xtest)
    scores.loc[j]=[j,k,i.score(xtrain,ytrain),accuracy_score(ytest,pred),recall_score(ytest,pred),precision_score(ytest,pred),confusion_matrix(ytest,pred)[0,0],confusion_matrix(ytest,pred)[0,1],confusion_matrix(ytest,pred)[1,1],confusion_matrix(ytest,pred)[1,0]]
    j=j+1
scores

,index,model,train score,accuracy score,recall score,precision score,tn,fp,tp,fn
0,0,Random forest,0.862620,0.766501,0.509091,0.583333,884,140,196,189
1,1,Bagging,0.858537,0.768630,0.514286,0.587537,885,139,198,187
2,2,Extra trees,0.862620,0.763662,0.462338,0.585526,898,126,178,207
3,3,adaboost,0.803514,0.791341,0.498701,0.655290,923,101,192,193
4,4,XG Boost,0.808129,0.789212,0.480519,0.656028,927,97,185,200
5,5,Logistic Regression,0.796947,0.794890,0.519481,0.657895,920,104,200,185
6,6,K Nearest Neighbors,0.821974,0.764372,0.467532,0.586319,897,127,180,205
7,7,Decision Tree,0.731274,0.752307,0.802597,0.530928,751,273,309,76
8,8,Gradient boosting,0.862620,0.770759,0.467532,0.604027,906,118,180,205


In [22]:
ch=pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")
ch.drop(["customerID"],axis=1,inplace=True)

ch["TotalCharges"][ch["TotalCharges"]==" "]=0
ch["TotalCharges"]=ch["TotalCharges"].astype("float64")

#ch["tenure"][ch["Contract"]=="Month-to-month"]=0
#ch["tenure"][ch["Contract"]=="One year"]=1
#ch["tenure"][ch["Contract"]=="Two year"]=2

C:\Users\Barath\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [23]:
#ch[["gender","Partner","Dependents","PhoneService",'PaperlessBilling',"Churn"]]=ch[["gender","Partner","Dependents","PhoneService",'PaperlessBilling',"Churn"]].apply(LabelEncoder().fit_transform)
ch[["gender","Partner","Dependents","PhoneService","MultipleLines","InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV",'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',"Churn"]]=ch[["gender","Partner","Dependents","PhoneService","MultipleLines","InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV",'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',"Churn"]].apply(LabelEncoder().fit_transform)



In [25]:
x=ch.drop(["Churn"],axis=1)
y=ch["Churn"]

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2)

In [26]:
scores=pd.DataFrame(columns=["index","model","train score","accuracy score","recall score","precision score","tn","fp","tp","fn"])
j=0
for i,k in zip(model,names):
    i.fit(xtrain,ytrain)
    pred=i.predict(xtest)
    scores.loc[j]=[j,k,i.score(xtrain,ytrain),accuracy_score(ytest,pred),recall_score(ytest,pred),precision_score(ytest,pred),confusion_matrix(ytest,pred)[0,0],confusion_matrix(ytest,pred)[0,1],confusion_matrix(ytest,pred)[1,1],confusion_matrix(ytest,pred)[1,0]]
    j=j+1
scores

C:\Users\Barath\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


,index,model,train score,accuracy score,recall score,precision score,tn,fp,tp,fn
0,0,Random forest,0.997870,0.797019,0.484694,0.693431,933,84,190,202
1,1,Bagging,0.981186,0.767211,0.418367,0.621212,917,100,164,228
2,2,Extra trees,0.997870,0.767921,0.438776,0.616487,910,107,172,220
3,3,adaboost,0.808662,0.795600,0.502551,0.679310,924,93,197,195
4,4,XG Boost,0.820731,0.797729,0.502551,0.686411,927,90,197,195
5,5,Logistic Regression,0.807774,0.779276,0.474490,0.639175,912,105,186,206
6,6,K Nearest Neighbors,0.833511,0.753726,0.413265,0.580645,900,117,162,230
7,7,Decision Tree,0.755769,0.757275,0.714286,0.549020,787,230,280,112
8,8,Gradient boosting,0.997870,0.729595,0.525510,0.513716,822,195,206,186
